<FONT size="5.9pt"> **CONSEQUENTIAL LCA OF BARK BIOREFINING WITH PRODUCTION OF HIGH STRENGTH MATERIALS VERSUS COMBUSTION (BASE CASE)**
</FONT> 

# 1. Set-up the directories <a class="anchor" id="section1"></a> 

It is advised to create a new folder in which the Jupyter Notebooks are downloaded from the GitHub repository and saved. A "results" folder will be created to store all outputs of the code (e.g., graphs, tables). This code uploads the ecoinvent database from a local directory. Therefore, a license for the ecoinvent is required and the database should be stored on your computer directly.

In [1]:
import os
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# define the initial working directory
wdir = Path().resolve() # get the directory of the notebook
pdir = os.path.join(wdir,'projects')
print(f"Working Directory: {wdir}")

# set the BW project directory
pathlib.Path(pdir).mkdir(parents=True, exist_ok=True)
os.environ['BRIGHTWAY2_DIR'] = pdir
print('Project directory:', os.environ['BRIGHTWAY2_DIR'])

# set the directory to store the results
result_dir = os.path.join(wdir,'results')   
pathlib.Path(result_dir).mkdir(parents=True, exist_ok=True)
print('Results directory:', result_dir)

# set the directory in which the databases are stored
db_dir = os.path.join(wdir,'databases')
pathlib.Path(db_dir).mkdir(parents=True, exist_ok=True)
print('Databases directory:', db_dir)

Working Directory: /home/leabraud/Documents/LCAbark
Project directory: /home/leabraud/Documents/LCAbark/projects
Results directory: /home/leabraud/Documents/LCAbark/results
Databases directory: /home/leabraud/Documents/LCAbark/databases


# 2. Import background databases <a class="anchor" id="section2"></a>

The background databases used in this study correspond to ecoinvent 3.9 consequential and biosphere3.

In [3]:
import lca_algebraic as alg
import brightway2 as bw

# initiate the project / choose project name
alg.initProject('lca_bark_sept25') ### NAME OF THE PROJECT CAN BE CHANGED

# import the ecoinvent database from a local directory
alg.importDb("ecoinvent_3.9_conseq", str(db_dir + '/ecoinvent 3.9_consequential_ecoSpold02/datasets')) ## VERSION OF ECOINVENT TO BE CHANGED

# overview of the databases
alg.list_databases()

Using environment variable BRIGHTWAY2_DIR for data directory:
/home/leabraud/Documents/LCAbark/projects


/tmp/ipykernel_58258/2000980516.py:5: DeprecationWarning: DEPRECATED : Use bw2io.import_ecoinvent_release() instead
  alg.initProject('lca_bark_sept25') ### NAME OF THE PROJECT CAN BE CHANGED
/tmp/ipykernel_58258/2000980516.py:8: DeprecationWarning: DEPRECATED : Use the new bw2io.import_ecoinvent_release instead
  alg.importDb("ecoinvent_3.9_conseq", str(db_dir + '/ecoinvent 3.9_consequential_ecoSpold02/datasets')) ## VERSION OF ECOINVENT TO BE CHANGED
[WARNING] Database 'ecoinvent_3.9_conseq' has already been imported 


Biosphere database already present!!! No setup is needed


,backend,nb_activities,type
name,,,
biosphere3,sqlite,4709,biosphere
ecoinvent_3.9_conseq,sqlite,18856,background
Foreground DB,sqlite,66,foreground


# 3. Create foreground databases <a class="anchor" id="section3"></a>

In [4]:
USER_DB = 'Foreground DB' ## NAME OF THE FOREGROUND DATABASE CAN BE CHANGED
alg.resetDb(USER_DB)
alg.resetParams()

[WARNING] Db Foreground DB was here. Reseting it


## 3.1. Get ecoinvent datasets

In [5]:
# Choose which version of ecoinvent to perform the LCA calculations with
ei = 'ecoinvent_3.9_conseq'

In [6]:
# technopshere flows
grid_SE = alg.findActivity(name='market for electricity, high voltage', loc = 'SE', db_name=ei) 
grid_RER = alg.findActivity(name='market group for electricity, high voltage', loc = 'RER', db_name=ei) 
ethylacetate = alg.findActivity(name='ethyl acetate production', loc = 'RER', db_name=ei) 
CO2 = alg.findActivity(name='carbon dioxide production, liquid', loc = 'RER', db_name=ei)
sodiumbicarbonate = alg.findActivity(name='market for sodium bicarbonate', loc = 'GLO', db_name=ei)
sodiumhydroxide = alg.findActivity(name='market for sodium hydroxide, without water, in 50% solution state', loc = 'GLO', db_name=ei)
quicklime = alg.findActivity(name='quicklime production, milled, loose', loc = 'CH', db_name=ei)
wwt = alg.findActivity(name='treatment of wastewater, average, wastewater treatment', loc = 'Europe without Switzerland', db_name=ei)
mw_RER = alg.findActivity(name='market group for municipal solid waste', loc = 'RER', db_name=ei) 
heatwoodchips = alg.findActivity(name='heat production, wood chips from industry, at furnace 50kW', loc = 'CH', db_name=ei)
tapwater = alg.findActivity(name='market group for tap water', loc = 'RER', db_name=ei)
hydrogenperoxide = alg.findActivity(name='market for hydrogen peroxide, without water, in 50% solution state', loc='RER', db_name=ei)
sodiumhypochlorite = alg.findActivity(name='market for sodium hypochlorite, without water, in 15% solution state', loc='RER', db_name=ei)
ozone = alg.findActivity(name='market for ozone, liquid', loc='RER', db_name=ei)

# substituted products
talloil = alg.findActivity(name='tall oil refinery operation', loc = 'GLO', db_name=ei) 
potatostarch = alg.findActivity(name='potato starch production', loc = 'DE', db_name=ei) 
phenol = alg.findActivity(name='phenol production, from cumene', loc = 'RER', db_name=ei)
pulp = alg.findActivity(name='sulfate pulp production, from softwood, unbleached', loc = 'RER', db_name=ei)
limestone = alg.findActivity(name='market for limestone residue', loc = 'CH', db_name=ei)
kraftliner = alg.findActivity(name='market for kraft paper', loc = 'RER', db_name=ei)
PET = alg.findActivity(name='polyethylene terephthalate production, granulate, bottle grade', loc = 'RER', db_name=ei)
LDPE = alg. findActivity(name='packaging film production, low density polyethylene', loc='RER', db_name=ei)

# biosphere flows
ethylacetateemission = alg.findActivity(name='Ethyl acetate', db_name='biosphere3', categories = ('air',))
water = alg.findActivity(name='Water, river', db_name='biosphere3', categories = ('natural resource','in water'))
carbondioxide = alg.findActivity(name='Carbon dioxide, fossil', db_name='biosphere3', categories = ('air',))
carbondioxidenonfossil = alg.findActivity(name='Carbon dioxide, non-fossil', db_name='biosphere3', categories = ('air',))

## 3.2. Modify the ecoinvent dataset for heat production via cogeneration

The ecoinvent dataset for heat production from wood chips was used to model the environmental burdens of marginal heat supply in both biorefinery configurations. It was also used to calculate the impacts of bark combustion in the combustion scenario and residual biomass combustion in the biorefineries. The dataset was adapted for each specific application (see below).

In [7]:
# dataset selected in ecoinvent
heat = alg.findActivity(name='heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', loc = 'SE', db_name=ei)

### 3.2.1. Combustion of bark 

The new dataset "heat_CHP_bark" includes the impacts associated with the cogeneration process itself, excluding those from wood chip production (assumed similar to bark). The system boundaries exclude bark production (as this process is similar in all three assessed scenarios). Since the properties of wood chips and bark are similar, it was assumed that combustion processes would result in the same impacts regardless of whether wood chips or bark are used as input.

In [8]:
heat_CHP_bark = alg.copyActivity(USER_DB, heat, code = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', withExchanges=True)
if ei == 'ecoinvent_3.9_conseq':
    heat_CHP_bark.deleteExchanges('wood chips, wet, measured as dry mass') # for ecoinvent 3.9
else:    
    heat_CHP_bark.deleteExchanges('market for wood chips, wet, measured as dry mass')

### 3.2.2. Burdens from marginal heat production

The unmodified ecoinvent dataset was used as a basis. Here, the objective is to model the heat production from wood chips that would be required to compensate for the heat produced when bark is combusted in the business-as-usual scenario (i.e. combustion scenario). The total amount of heat that would need to be produced for compensation corresponds to the total amount of heat resulting from the combustion of 1 kg DW-eq of bark minus the heat produced via combustion of the residual biomass obtained in the biorefinery. The original process extracted from ecoinvent is not changed since the impacts from wood chips production and combustion are accounted for. The original activity is simply copied and given a new name for distinction.

In [9]:
heat_CHP_woodchips = alg.copyActivity(USER_DB, heat, code = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', withExchanges=True)

[WARNING] Activity 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' was already in 'Foreground DB'. Overwriting it


### 3.2.3. Heat production from residual biomass

Again, the same ecoinvent dataset is used as a basis. The amount of residual biomass produced at each processing step is calculated from the mass and carbon balance. The amount of heat produced is calculated using the relation: 1 kg dry mass => 8.5 MJ according to the ecoinvent dataset for heat from cogeneration using wood chips. The same dataset as the one used for heat production from bark is used. No impacts from the biomass are accounted for. A new name is given for distinction. 

In [10]:
heat_CHP_RB = alg.copyActivity(USER_DB, heat, code = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', withExchanges=True)
if ei == 'ecoinvent_3.9_conseq':
    heat_CHP_RB.deleteExchanges('wood chips, wet, measured as dry mass') # ecoinvent 3.9
else:
    heat_CHP_RB.deleteExchanges('market for wood chips, wet, measured as dry mass')

[WARNING] Activity 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' was already in 'Foreground DB'. Overwriting it


In [11]:
# check the climate impacts of heat production using IPCC 2021
ipcc2021 = alg.findMethods('global warming potential (GWP100)', 'IPCC 2021')[0]
climate_impacts_heat_CHP_RB = alg.compute_impacts([heat_CHP_RB],[ipcc2021])
climate_impacts_heat_CHP_RB

[INFO] Db changed recently, clearing cache expr
[INFO] Db changed recently, clearing cache lcia


,climate change - global warming potential (GWP100)[kg CO2-Eq]
"heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014[SE]",-0.0142485


In [12]:
recipeGWP = alg.findMethods('ReCiPe 2016 v1.03, midpoint (H)')[19]
climate_impacts_heat_CHP_RB_recipe = alg.compute_impacts([heat_CHP_RB],[recipeGWP])
climate_impacts_heat_CHP_RB_recipe

[INFO] Db changed recently, clearing cache expr


,climate change - global warming potential (GWP1000)[kg CO2-Eq]
"heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014[SE]",0.177779


# 4. Create foreground parameters <a class="anchor" id="section4"></a>

A foreground parameter is created to evaluate the influence of the integration of the biorefinery with co-located source of energy. The boolean parameter defined changes the amount of electricity used in the biorefinery processes. In the integrated biorefinery (standalone = 0), no external electricity is required and all the energy is supplied from the co-located source which comes into the system free of burdens. In the standalone biorefinery (standalone = 1), the electricity required is supplied by the grid. 

In [13]:
standalone = alg.newBoolParam('standalone', 
                               default = 1, # 1 means the biorefinery system is stand-alone, 0 it is integrated
                               group = 'booleans')

# 5. Add the parameters to the foreground activities <a class="anchor" id="section5"></a>

## 5.1. Create parameterised biorefinery processes

All the activities are scaled to the functional unit i.e. the processing of 1 kg of bark dry mass. 

In [14]:
# lipophilic extraction from bark
lipophilic_extraction = alg.newActivity(db_name = USER_DB, 
         name = 'lipophilic extraction',
         unit = 'unit',
         exchanges={
             # technosphere inputs
             ethylacetate: 0.002,
             grid_SE: 2.44*standalone,
             # emissions
             ethylacetateemission: 0.002
         })
alg.printAct(lipophilic_extraction)

lipophilic extraction (1.000000 unit)  \
                                                                         input   
Ethyl acetate                                                    Ethyl acetate   
ethyl acetate production                         ethyl acetate production[RER]   
market for electricity, high voltage  market for electricity, high voltage[SE]   

                                                                      
                                               amount           unit  
Ethyl acetate                                   0.002       kilogram  
ethyl acetate production                        0.002       kilogram  
market for electricity, high voltage  2.44*standalone  kilowatt hour

In [15]:
# sugar extraction from lipophilic depleted bark
sugar_extraction = alg.newActivity(db_name = USER_DB, 
 name = 'sugar extraction',
 unit = 'unit',
 exchanges={
     grid_SE: 5*standalone,
     wwt: -0.00007,
     # natural resources
     water: 0.00007
 })
alg.printAct(sugar_extraction)

sugar extraction (1.000000 unit)  \
                                                                                                input   
Water, river                                                                             Water, river   
market for electricity, high voltage                         market for electricity, high voltage[SE]   
treatment of wastewater, average, wastewater tr...  treatment of wastewater, average, wastewater t...   

                                                                  \
                                                          amount   
Water, river                                               7e-05   
market for electricity, high voltage                5*standalone   
treatment of wastewater, average, wastewater tr...        -7e-05   

                                                                   
                                                             unit  
Water, river                                          cubic meter  
market for electricity, high voltage                kilowatt hour  
treatment of wastewater, average, wastewater tr...    cubic meter

In [16]:
# pulping of sugar depleted bark
pulping = alg.newActivity(db_name = USER_DB, 
 name = 'pulping',
 unit = 'unit',
 exchanges={
     sodiumhydroxide: 0.00523,
     quicklime: 0.00629, # calcium oxide
     wwt: -0.00035,
     grid_SE: 1.03*standalone,
     heatwoodchips:0.35,
     CO2: 0.124*standalone,
     limestone:-0.00551,
     # natural resources
     water: 0.00035
 })
alg.printAct(pulping)

pulping (1.000000 unit)  \
                                                                                                input   
Water, river                                                                             Water, river   
carbon dioxide production, liquid                              carbon dioxide production, liquid[RER]   
heat production, wood chips from industry, at f...  heat production, wood chips from industry, at ...   
market for electricity, high voltage                         market for electricity, high voltage[SE]   
market for limestone residue                                         market for limestone residue[CH]   
market for sodium hydroxide, without water, in ...  market for sodium hydroxide, without water, in...   
quicklime production, milled, loose                           quicklime production, milled, loose[CH]   
treatment of wastewater, average, wastewater tr...  treatment of wastewater, average, wastewater t...   

                                                                      \
                                                              amount   
Water, river                                                 0.00035   
carbon dioxide production, liquid                   0.124*standalone   
heat production, wood chips from industry, at f...              0.35   
market for electricity, high voltage                 1.03*standalone   
market for limestone residue                                -0.00551   
market for sodium hydroxide, without water, in ...           0.00523   
quicklime production, milled, loose                          0.00629   
treatment of wastewater, average, wastewater tr...          -0.00035   

                                                                   
                                                             unit  
Water, river                                          cubic meter  
carbon dioxide production, liquid                        kilogram  
heat production, wood chips from industry, at f...      megajoule  
market for electricity, high voltage                kilowatt hour  
market for limestone residue                             kilogram  
market for sodium hydroxide, without water, in ...       kilogram  
quicklime production, milled, loose                      kilogram  
treatment of wastewater, average, wastewater tr...    cubic meter

In [17]:
# coated Lig-MFC (70 MPa)
coatedLigMFC = alg.newActivity(db_name = USER_DB, 
 name = 'coatedLigMFC',
 unit = 'unit',
 exchanges={
     tapwater: 0.55,
     grid_SE: 2.175*standalone,
 })
alg.printAct(coatedLigMFC)

coatedLigMFC (1.000000 unit)  \
                                                                         input   
market for electricity, high voltage  market for electricity, high voltage[SE]   
market group for tap water                     market group for tap water[RER]   

                                                                       
                                                amount           unit  
market for electricity, high voltage  2.175*standalone  kilowatt hour  
market group for tap water                        0.55       kilogram

In [18]:
# coatedbleachMFC (170 MPa)
bleachcoatMFC = alg.newActivity(db_name = USER_DB, 
 name = 'bleachcoatMFC',
 unit = 'unit',
 exchanges={
     hydrogenperoxide:0.024,
     ozone:0.066*0.059-0.0176,
     sodiumhypochlorite:0.215*0.059-0.0293,
     tapwater:0.28,
     grid_SE: 2.32*standalone,
 })
alg.printAct(bleachcoatMFC)

bleachcoatMFC (1.000000 unit)  \
                                                                                                input   
market for electricity, high voltage                         market for electricity, high voltage[SE]   
market for hydrogen peroxide, without water, in...  market for hydrogen peroxide, without water, i...   
market for ozone, liquid                                                market for ozone, liquid[RER]   
market for sodium hypochlorite, without water, ...  market for sodium hypochlorite, without water,...   
market group for tap water                                            market group for tap water[RER]   

                                                                     \
                                                             amount   
market for electricity, high voltage                2.32*standalone   
market for hydrogen peroxide, without water, in...            0.024   
market for ozone, liquid                                  -0.013706   
market for sodium hypochlorite, without water, ...        -0.016615   
market group for tap water                                     0.28   

                                                                   
                                                             unit  
market for electricity, high voltage                kilowatt hour  
market for hydrogen peroxide, without water, in...       kilogram  
market for ozone, liquid                                 kilogram  
market for sodium hypochlorite, without water, ...       kilogram  
market group for tap water                               kilogram

In [19]:
# production of heat from residual biomass via cogeneration
heat_residues = alg.newActivity(db_name = USER_DB, 
 name = 'heat (RB)',
 unit = 'unit',
 exchanges={
     heat_CHP_RB: 1.53
 })
alg.printAct(heat_residues)

heat (RB) (1.000000 unit)  \
                                                                                                input   
heat and power co-generation, wood chips, 6667 ...  heat and power co-generation, wood chips, 6667...   

                                                                      
                                                   amount       unit  
heat and power co-generation, wood chips, 6667 ...   1.53  megajoule

## 5.3. Create parameterised substituted processes

In [20]:
# substitution of tall oil from pine wood (1:1) (lipophilic extractives from lipophilic extraction)
talloil_sub = alg.newActivity(db_name = USER_DB,
                             name = 'tall oil (sub.)',
                             unit = 'unit',
                             exchanges = {talloil: - ((0.02/0.82)*(1-0.18))})
alg.printAct(talloil_sub)

tall oil (sub.) (1.000000 unit)                 
                                                      input amount      unit
tall oil refinery operation     tall oil refinery operation  -0.02  kilogram

In [21]:
# substitution of starch from potato (1:1) (hydrophilic extractives from sugar extraction)
starch_sub = alg.newActivity(db_name = USER_DB,
                          name = 'starch (sub.)',
                          unit = 'unit',
                          exchanges = {potatostarch: -((0.22/0.82)*(1-0.18))})
alg.printAct(starch_sub)

starch (sub.) (1.000000 unit)                 
                                                 input amount      unit
potato starch production  potato starch production[DE]  -0.22  kilogram

In [22]:
# substitution of phenol from cumene (1:1) (phenol from pulping)
phenol_sub = alg.newActivity(db_name = USER_DB,
                            name = 'phenol (sub.)',
                            unit = 'unit',
                            exchanges = {phenol: -((0.17/0.82)*(1-0.18))})
alg.printAct(phenol_sub)

phenol (sub.) (1.000000 unit)         \
                                                              input amount   
phenol production, from cumene  phenol production, from cumene[RER]  -0.17   

                                          
                                    unit  
phenol production, from cumene  kilogram

In [23]:
# substitution of sulfate pulp (1:1) (pulp from pulping)
pulp_sub = alg.newActivity(db_name = USER_DB,
                            name = 'pulp (sub.)',
                            unit = 'unit',
                            exchanges = {pulp: -((0.41/0.82)*(1-0.18))})
alg.printAct(pulp_sub)

pulp (sub.) (1.000000 unit)  \
                                                                                                input   
sulfate pulp production, from softwood, unbleached  sulfate pulp production, from softwood, unblea...   

                                                                     
                                                   amount      unit  
sulfate pulp production, from softwood, unbleached  -0.41  kilogram

In [24]:
# environmental impacts from the production of marginal heat in the biorefinery scenarios (e.g., heat that would have been produced by directly burning the bark)
# amount corresponds to the total amount of heat produced when burning bark (8.5 MJ) minus the heat produced from the residues
heat_burdens = alg.newActivity(db_name = USER_DB, 
 name = 'heat (burd.)',
 unit = 'unit',
 exchanges={
     heat_CHP_woodchips: 8.5 -(1.53)
 })
alg.printAct(heat_burdens)

heat (burd.) (1.000000 unit)  \
                                                                                                input   
heat and power co-generation, wood chips, 6667 ...  heat and power co-generation, wood chips, 6667...   

                                                                      
                                                   amount       unit  
heat and power co-generation, wood chips, 6667 ...   6.97  megajoule

In [25]:
# substitution of kraft liner by coatLigMF
kraftliner_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                            name = 'Kraft kiner coatLigMF (sub.)',
                            unit = 'unit',
                            exchanges = {kraftliner: -0.99})
alg.printAct(kraftliner_sub_coatLigMF)

Kraft kiner coatLigMF (sub.) (1.000000 unit)         \
                                                              input amount   
market for kraft paper                  market for kraft paper[RER]  -0.99   

                                  
                            unit  
market for kraft paper  kilogram

In [26]:
# substitution of PET by coatLigMF
PET_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                            name = 'PET coatLigMF (sub.)',
                            unit = 'unit',
                            exchanges = {PET: -0.46})
alg.printAct(PET_sub_coatLigMF)

PET coatLigMF (sub.) (1.000000 unit)  \
                                                                                                input   
polyethylene terephthalate production, granulat...  polyethylene terephthalate production, granula...   

                                                                     
                                                   amount      unit  
polyethylene terephthalate production, granulat...  -0.46  kilogram

In [27]:
# substitution of LDPE by coatLigMF
LDPE_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                            name = 'LDPE coatLigMF (sub.)',
                            unit = 'unit',
                            exchanges = {LDPE: -2.44})
alg.printAct(LDPE_sub_coatLigMF)

LDPE coatLigMF (sub.) (1.000000 unit)  \
                                                                                                input   
packaging film production, low density polyethy...  packaging film production, low density polyeth...   

                                                                     
                                                   amount      unit  
packaging film production, low density polyethy...  -2.44  kilogram

In [28]:
# substitution of sulfate pulp (1:1) in coatLigMF scenario (0.41-0.39 = 0.02 kg)
pulp_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                            name = 'pulp coatLigMF (sub.)',
                            unit = 'unit',
                            exchanges = {pulp: -((0.02/0.82)*(1-0.18))})
alg.printAct(pulp_sub_coatLigMF)

pulp coatLigMF (sub.) (1.000000 unit)  \
                                                                                                input   
sulfate pulp production, from softwood, unbleached  sulfate pulp production, from softwood, unblea...   

                                                                     
                                                   amount      unit  
sulfate pulp production, from softwood, unbleached  -0.02  kilogram

In [29]:
# substitution of starch from potato (1:1) in coatLigMF scenario (0.22-0.19 = 0.03 kg)
starch_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                          name = 'starch coatLigMF (sub.)',
                          unit = 'unit',
                          exchanges = {potatostarch: -((0.03/0.82)*(1-0.18))})
alg.printAct(starch_sub_coatLigMF)

starch coatLigMF (sub.) (1.000000 unit)         \
                                                           input amount   
potato starch production            potato starch production[DE]  -0.03   

                                    
                              unit  
potato starch production  kilogram

In [30]:
# substitution of kraft liner by bleachcoatMFC
kraftliner_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                            name = 'Kraft kiner bleachcoatMFC (sub.)',
                            unit = 'unit',
                            exchanges = {kraftliner: -1.12})
alg.printAct(kraftliner_sub_bleachcoatMFC)

Kraft kiner bleachcoatMFC (sub.) (1.000000 unit)  \
                                                                  input   
market for kraft paper                      market for kraft paper[RER]   

                                         
                       amount      unit  
market for kraft paper  -1.12  kilogram

In [31]:
# substitution of PET bleachcoatMFC
PET_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                            name = 'PET bleachcoatMFC (sub.)',
                            unit = 'unit',
                            exchanges = {PET: -0.53})
alg.printAct(PET_sub_bleachcoatMFC)

PET bleachcoatMFC (sub.) (1.000000 unit)  \
                                                                                                input   
polyethylene terephthalate production, granulat...  polyethylene terephthalate production, granula...   

                                                                     
                                                   amount      unit  
polyethylene terephthalate production, granulat...  -0.53  kilogram

In [32]:
# substitution of LDPE by bleachcoatMFC
LDPE_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                            name = 'LDPE bleachcoatMFC (sub.)',
                            unit = 'unit',
                            exchanges = {LDPE: -2.67})
alg.printAct(LDPE_sub_bleachcoatMFC)

LDPE bleachcoatMFC (sub.) (1.000000 unit)  \
                                                                                                input   
packaging film production, low density polyethy...  packaging film production, low density polyeth...   

                                                                     
                                                   amount      unit  
packaging film production, low density polyethy...  -2.67  kilogram

In [33]:
# substitution of heat by residues from bleachcoatMFC process
heat_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                            name = 'heat bleachcoatMFC (sub.)',
                            unit = 'unit',
                            exchanges = {heat_CHP_woodchips: -1.74})
alg.printAct(heat_sub_bleachcoatMFC)

heat bleachcoatMFC (sub.) (1.000000 unit)  \
                                                                                                input   
heat and power co-generation, wood chips, 6667 ...  heat and power co-generation, wood chips, 6667...   

                                                                      
                                                   amount       unit  
heat and power co-generation, wood chips, 6667 ...  -1.74  megajoule

In [34]:
# substitution of starch from potato (1:1) in coatLigMF scenario (0.22-0.076 = 0.144)
starch_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                          name = 'starch bleachcoatMFC (sub.)',
                          unit = 'unit',
                          exchanges = {potatostarch: -((0.144/0.82)*(1-0.18))})
alg.printAct(starch_sub_bleachcoatMFC)

starch bleachcoatMFC (sub.) (1.000000 unit)         \
                                                               input amount   
potato starch production                potato starch production[DE] -0.144   

                                    
                              unit  
potato starch production  kilogram

## 5.4. Scaled process for combustion scenario

In [35]:
heat_CHP_bark_scaled = alg.newActivity(db_name = USER_DB,
                                     name = 'heat CHP bark',
                                     unit = 'unit',
                                     exchanges={heat_CHP_bark:8.5})

# 6. Calculate the static climate impacts using IPCC 2021 GWP100

In [36]:
df_bioref_standalone_baseline = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub,
                                            starch_sub],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                          )

[INFO] Db changed recently, clearing cache expr
[INFO] Db changed recently, clearing cache lcia
[INFO] Db changed recently, clearing cache expr
[INFO] Db changed recently, clearing cache lcia
[INFO] Db changed recently, clearing cache expr


In [37]:
df_bioref_standalone_coatedLigMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            kraftliner_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                          )

In [38]:
df_bioref_standalone_coatedLigMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            PET_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                          )

In [39]:
df_bioref_standalone_coatedLigMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            LDPE_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1
                                          )

In [40]:
df_bioref_standalone_bleachcoatMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            kraftliner_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1
                                          )

In [41]:
df_bioref_standalone_bleachcoatMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            PET_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1
                                          )

In [42]:
df_bioref_standalone_bleachcoatMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            LDPE_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1
                                          )

In [43]:
df_bioref_integrated_baseline = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub,
                                            starch_sub],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0,
                                          )

In [44]:
df_bioref_integrated_coatedLigMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            kraftliner_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0
                                          )

In [45]:
df_bioref_integrated_coatedLigMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            PET_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0
                                          )

In [46]:
df_bioref_integrated_coatedLigMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            LDPE_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0
                                          )

In [47]:
df_bioref_integrated_bleachcoatMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            kraftliner_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0
                                          )

In [48]:
df_bioref_integrated_bleachcoatMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            PET_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0
                                          )

In [49]:
df_bioref_integrated_bleachcoatMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            LDPE_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0
                                          )

In [50]:
# static climate impacts (indirect emissions only) for the combustion scenario
df_combustion = alg.compute_impacts([heat_CHP_bark_scaled],
                            [ipcc2021]
                            )
#df_combustion.to_excel(str(result_dir+"/climate_impacts_IPCC2021_static_combustion_" + str(ei)+ ".xlsx"))

In [51]:
df_bioref_standalone_baseline

,climate change - global warming potential (GWP100)[kg CO2-Eq]
lipophilic extraction,0.449329
sugar extraction,0.901768
pulping,0.353132
heat (burd.),-0.0993123
heat (RB),-0.0218003
tall oil (sub.),-0.0366491
phenol (sub.),-0.401767
pulp (sub.),0.0279059
starch (sub.),-0.369547


In [52]:
# merge the three dataframes
merged_df = pd.concat([df_combustion, 
                       df_bioref_standalone_baseline,                        
                       df_bioref_standalone_coatedLigMFC_KL, 
                       df_bioref_standalone_coatedLigMFC_PET, 
                       df_bioref_standalone_coatedLigMFC_LDPE, 
                       df_bioref_standalone_bleachcoatMFC_KL, 
                       df_bioref_standalone_bleachcoatMFC_PET, 
                       df_bioref_standalone_bleachcoatMFC_LDPE, 
                       df_bioref_integrated_baseline, 
                       df_bioref_integrated_coatedLigMFC_KL, 
                       df_bioref_integrated_coatedLigMFC_PET, 
                       df_bioref_integrated_coatedLigMFC_LDPE, 
                       df_bioref_integrated_bleachcoatMFC_KL, 
                       df_bioref_integrated_bleachcoatMFC_PET, 
                       df_bioref_integrated_bleachcoatMFC_LDPE, 
                       ], 
                      axis = 1)
# change the name of the columns
merged_df.columns = ['Combustion',
                     'Standalone baseline',
                     'Standalone coatedLigMFC KL',
                     'Standalone coatedLigMFC PET',
                     'Standalone coatedLigMFC LDPE',
                     'Standalone bleachcoatMFC KL',
                     'Standalone bleachcoatMFC PET',
                     'Standalone bleachcoatMFC LDPE',
                     'Integrated baseline',
                     'Integrated coatedLigMFC KL',
                     'Integrated coatedLigMFC PET',
                     'Integrated coatedLigMFC LDPE',
                     'Integrated bleachcoatMFC KL',
                     'Integrated bleachcoatMFC PET',
                     'Integrated bleachcoatMFC LDPE',
                    ]
merged_df

,Combustion,Standalone baseline,Standalone coatedLigMFC KL,Standalone coatedLigMFC PET,Standalone coatedLigMFC LDPE,Standalone bleachcoatMFC KL,Standalone bleachcoatMFC PET,Standalone bleachcoatMFC LDPE,Integrated baseline,Integrated coatedLigMFC KL,Integrated coatedLigMFC PET,Integrated coatedLigMFC LDPE,Integrated bleachcoatMFC KL,Integrated bleachcoatMFC PET,Integrated bleachcoatMFC LDPE
heat CHP bark,-0.121113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lipophilic extraction,NaN,0.449329,0.449329,0.449329,0.449329,0.449329,0.449329,0.449329,0.00927422,0.00927422,0.00927422,0.00927422,0.00927422,0.00927422,0.00927422
sugar extraction,NaN,0.901768,0.901768,0.901768,0.901768,0.901768,0.901768,0.901768,1.6207e-05,1.6207e-05,1.6207e-05,1.6207e-05,1.6207e-05,1.6207e-05,1.6207e-05
pulping,NaN,0.353132,0.353132,0.353132,0.353132,0.353132,0.353132,0.353132,0.034189,0.034189,0.034189,0.034189,0.034189,0.034189,0.034189
heat (burd.),NaN,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123,-0.0993123
heat (RB),NaN,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003,-0.0218003
tall oil (sub.),NaN,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491
phenol (sub.),NaN,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767
pulp (sub.),NaN,0.0279059,NaN,NaN,NaN,NaN,NaN,NaN,0.0279059,NaN,NaN,NaN,NaN,NaN,NaN
starch (sub.),NaN,-0.369547,NaN,NaN,NaN,NaN,NaN,NaN,-0.369547,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
merged_df.loc["Total"] = merged_df.sum()

In [54]:
merged_df.to_excel('LCIA_results_climate_change.xlsx')

# 7. Calculate other impacts using the ReCiPe LCIA methods

In [55]:
recipe2016 = alg.findMethods('ReCiPe 2016 v1.03, midpoint (H)')
recipe2016 = [method for method in recipe2016 if 'no LT' not in str(method)]
recipe2016 = [method for method in recipe2016 if 'climate change' not in str(method)]
recipe2016
len(recipe2016)

17

In [56]:
df_bioref_standalone_baseline = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub,
                                            starch_sub],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 1,
                                          )
total_df_bioref_standalone_baseline = pd.DataFrame(df_bioref_standalone_baseline.sum(axis=0)).T
total_df_bioref_standalone_baseline.index = ['Standalone baseline']

In [57]:
df_bioref_standalone_coatedLigMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            kraftliner_sub_coatLigMF],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 1
                                          )
total_df_bioref_standalone_coatedLigMFC_KL = pd.DataFrame(df_bioref_standalone_coatedLigMFC_KL.sum(axis=0)).T
total_df_bioref_standalone_coatedLigMFC_KL.index = ['Standalone coatedLigMFC KL']

In [58]:
df_bioref_standalone_coatedLigMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            PET_sub_coatLigMF],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 1
                                          )
total_df_bioref_standalone_coatedLigMFC_PET = pd.DataFrame(df_bioref_standalone_coatedLigMFC_PET.sum(axis=0)).T
total_df_bioref_standalone_coatedLigMFC_PET.index = ['Standalone coatedLigMFC PET']

In [59]:
df_bioref_standalone_coatedLigMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            LDPE_sub_coatLigMF],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 1
                                          )
total_df_bioref_standalone_coatedLigMFC_LDPE = pd.DataFrame(df_bioref_standalone_coatedLigMFC_LDPE.sum(axis=0)).T
total_df_bioref_standalone_coatedLigMFC_LDPE.index = ['Standalone bleachcoatMFC LDPE']

In [60]:
df_bioref_standalone_bleachcoatMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            kraftliner_sub_bleachcoatMFC],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 1
                                          )
total_df_bioref_standalone_bleachcoatMFC_KL = pd.DataFrame(df_bioref_standalone_bleachcoatMFC_KL.sum(axis=0)).T
total_df_bioref_standalone_bleachcoatMFC_KL.index = ['Standalone bleachcoatMFC KL']

In [61]:
df_bioref_standalone_bleachcoatMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            PET_sub_bleachcoatMFC],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 1
                                          )
total_df_bioref_standalone_bleachcoatMFC_PET = pd.DataFrame(df_bioref_standalone_bleachcoatMFC_PET.sum(axis=0)).T
total_df_bioref_standalone_bleachcoatMFC_PET.index = ['Standalone bleachcoatMFC PET']

In [62]:
df_bioref_standalone_bleachcoatMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            LDPE_sub_bleachcoatMFC],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 1
                                          )
total_df_bioref_standalone_bleachcoatMFC_LDPE = pd.DataFrame(df_bioref_standalone_bleachcoatMFC_LDPE.sum(axis=0)).T
total_df_bioref_standalone_bleachcoatMFC_LDPE.index = ['Standalone bleachcoatMFC LDPE']

In [63]:
df_bioref_integrated_baseline = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub,
                                            starch_sub],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 0,
                                          )
total_df_bioref_integrated_baseline = pd.DataFrame(df_bioref_integrated_baseline.sum(axis=0)).T
total_df_bioref_integrated_baseline.index = ['Integrated baseline']

In [64]:
df_bioref_integrated_coatedLigMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            kraftliner_sub_coatLigMF],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 0
                                          )
total_df_bioref_integrated_coatedLigMFC_KL = pd.DataFrame(df_bioref_integrated_coatedLigMFC_KL.sum(axis=0)).T
total_df_bioref_integrated_coatedLigMFC_KL.index = ['Integrated coatedLigMFC KL']

In [65]:
df_bioref_integrated_coatedLigMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            PET_sub_coatLigMF],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 0
                                          )
total_df_bioref_integrated_coatedLigMFC_PET = pd.DataFrame(df_bioref_integrated_coatedLigMFC_PET.sum(axis=0)).T
total_df_bioref_integrated_coatedLigMFC_PET.index = ['Integrated coatedLigMFC PET']

In [66]:
df_bioref_integrated_coatedLigMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            LDPE_sub_coatLigMF],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 0
                                          )
total_df_bioref_integrated_coatedLigMFC_LDPE = pd.DataFrame(df_bioref_integrated_coatedLigMFC_LDPE.sum(axis=0)).T
total_df_bioref_integrated_coatedLigMFC_LDPE.index = ['Integrated coatedLigMFC LDPE']

In [67]:
df_bioref_integrated_bleachcoatMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            kraftliner_sub_bleachcoatMFC],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 0
                                          )
total_df_bioref_integrated_bleachcoatMFC_KL = pd.DataFrame(df_bioref_integrated_bleachcoatMFC_KL.sum(axis=0)).T
total_df_bioref_integrated_bleachcoatMFC_KL.index = ['Integrated bleachcoatMFC KL']

In [68]:
df_bioref_integrated_bleachcoatMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            PET_sub_bleachcoatMFC],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 0
                                          )
total_df_bioref_integrated_bleachcoatMFC_PET = pd.DataFrame(df_bioref_integrated_bleachcoatMFC_PET.sum(axis=0)).T
total_df_bioref_integrated_bleachcoatMFC_PET.index = ['Integrated bleachcoatMFC PET']

In [69]:
df_bioref_integrated_bleachcoatMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction, 
                                            pulping, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            LDPE_sub_bleachcoatMFC],
                                            # lcia method
                                            recipe2016,
                                            # parameters
                                            standalone = 0
                                          )
total_df_bioref_integrated_bleachcoatMFC_LDPE = pd.DataFrame(df_bioref_integrated_bleachcoatMFC_LDPE.sum(axis=0)).T
total_df_bioref_integrated_bleachcoatMFC_LDPE.index = ['Integrated bleachcoatMFC LDPE']

In [70]:
# static climate impacts (indirect emissions only) for the combustion scenario
df_combustion = alg.compute_impacts([heat_CHP_bark_scaled],
                            recipe2016
                            )
total_df_combustion = pd.DataFrame(df_combustion.sum(axis=0)).T
total_df_combustion.index = ['Combustion']
#df_combustion.to_excel(str(result_dir+"/climate_impacts_IPCC2021_static_combustion_" + str(ei)+ ".xlsx"))

In [71]:
merged_df_allcat = pd.concat([total_df_combustion, 
                       total_df_bioref_standalone_baseline,                              
                       total_df_bioref_standalone_coatedLigMFC_KL,
                       total_df_bioref_standalone_coatedLigMFC_PET,                       
                       total_df_bioref_standalone_coatedLigMFC_LDPE,
                       total_df_bioref_standalone_bleachcoatMFC_KL,
                       total_df_bioref_standalone_bleachcoatMFC_PET,
                       total_df_bioref_standalone_bleachcoatMFC_LDPE,
                       total_df_bioref_integrated_baseline,
                       total_df_bioref_integrated_coatedLigMFC_KL,
                       total_df_bioref_integrated_coatedLigMFC_PET,                       
                       total_df_bioref_integrated_coatedLigMFC_LDPE,
                       total_df_bioref_integrated_bleachcoatMFC_KL,
                       total_df_bioref_integrated_bleachcoatMFC_PET,
                       total_df_bioref_integrated_bleachcoatMFC_LDPE
                      ])
merged_df_allcat

,acidification: terrestrial - terrestrial acidification potential (TAP)[kg SO2-Eq],"ecotoxicity: freshwater - freshwater ecotoxicity potential (FETP)[kg 1,4-DCB-Eq]","ecotoxicity: marine - marine ecotoxicity potential (METP)[kg 1,4-DCB-Eq]","ecotoxicity: terrestrial - terrestrial ecotoxicity potential (TETP)[kg 1,4-DCB-Eq]","energy resources: non-renewable, fossil - fossil fuel potential (FFP)[kg oil-Eq]",eutrophication: freshwater - freshwater eutrophication potential (FEP)[kg P-Eq],eutrophication: marine - marine eutrophication potential (MEP)[kg N-Eq],"human toxicity: carcinogenic - human toxicity potential (HTPc)[kg 1,4-DCB-Eq]","human toxicity: non-carcinogenic - human toxicity potential (HTPnc)[kg 1,4-DCB-Eq]",ionising radiation - ionising radiation potential (IRP)[kg Co-60-Eq],land use - agricultural land occupation (LOP)[m2*a crop-Eq],material resources: metals/minerals - surplus ore potential (SOP)[kg Cu-Eq],ozone depletion - ozone depletion potential (ODPinfinite)[kg CFC-11-Eq],particulate matter formation - particulate matter formation potential (PMFP)[kg PM2.5-Eq],photochemical oxidant formation: human health - photochemical oxidant formation potential: humans (HOFP)[kg NOx-Eq],photochemical oxidant formation: terrestrial ecosystems - photochemical oxidant formation potential: ecosystems (EOFP)[kg NOx-Eq],water use - water consumption potential (WCP)[cubic meter]
Combustion,0.0010244,-0.0106921,-0.0119435,0.734945,-0.0536851,-1.10268e-06,-6.1386e-07,-0.00153805,0.304767,-0.000279757,-0.0826799,0.000103424,3.48945e-07,0.000284412,0.00123649,0.00123925,0.000734144
Standalone baseline,-0.000691854,0.114064,0.154227,3.67051,0.251152,0.000114795,-0.000815191,0.0246412,0.651293,0.00217663,-0.523994,0.000786972,-4.42236e-07,0.000247764,0.002416,0.00264467,-0.070212
Standalone coatedLigMFC KL,0.000868005,0.146363,0.180858,5.12983,0.327413,-0.00158092,-0.000431585,-0.017256,0.875565,-0.0435159,-1.28631,0.00174114,1.28252e-06,0.000282212,0.00222915,0.00234946,-0.021874
Standalone coatedLigMFC PET,0.000561174,0.126851,0.157029,7.13846,-0.223468,-0.000115162,-0.000190855,-0.0109941,0.467023,-0.0492358,0.910514,-0.00853919,-7.12151e-06,-0.000331682,0.00253886,0.00267077,-0.0325001
Standalone bleachcoatMFC LDPE,-0.0125924,0.0168289,0.00858944,-7.91278,-3.90098,-0.00137062,-0.000356446,-0.146941,-2.11873,-0.307763,0.664261,-0.00363122,8.01513e-07,-0.00695227,-0.00944269,-0.0108018,-0.119261
Standalone bleachcoatMFC KL,-0.00176717,0.127623,0.163062,2.28788,0.267929,-0.00195217,-0.000896829,-0.0284756,0.211398,-0.0696737,-1.70737,0.000332925,-9.75246e-08,-0.000436615,0.000846912,0.000912881,-0.0585467
Standalone bleachcoatMFC PET,-0.00216449,0.104515,0.134774,4.5176,-0.370131,-0.000300386,-0.000625581,-0.0223683,-0.273377,-0.0772318,0.777356,-0.0115538,-9.79192e-06,-0.0011617,0.00114464,0.00121989,-0.0710254
Standalone bleachcoatMFC LDPE,-0.0164186,-0.0130088,-0.0239656,-11.8339,-4.3531,-0.00165628,-0.00080375,-0.168418,-3.04017,-0.357111,0.509476,-0.00547122,-6.03466e-07,-0.00832147,-0.0118202,-0.0133658,-0.164696
Integrated baseline,-0.00323334,-0.0405289,-0.0367399,-4.28079,-0.386407,-2.03559e-05,-0.000831791,-0.0112291,-0.490061,0.00191669,-1.41127,-0.00253922,-1.95478e-06,-0.000700015,-0.0010043,-0.000870483,-0.0753474
Integrated coatedLigMFC KL,-0.00229216,-0.0476435,-0.0587128,-4.63054,-0.465134,-0.00174238,-0.00045082,-0.0616548,-0.538501,-0.0439957,-2.40272,-0.0024056,-6.15593e-07,-0.000891619,-0.00203949,-0.00203553,-0.0282143


In [72]:
merged_df_allcat.to_excel('LCIA_results_allcat.xlsx', index=True)